In [65]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [67]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [70]:
!echo "# MachineLearning" >> README.md
!git init

Reinitialized existing Git repository in /content/drive/MyDrive/Colab Notebooks/.git/


In [76]:
!git config --global user.email "hyobin021218@gmail.com"
!git config --global user.name "dohb128"

In [77]:
# 1. 파일 변경 사항이 스테이징되어 있다고 가정하고 바로 커밋을 시도합니다.
# (이전에 !git add README.md 를 실행했다면)
!git commit -m "first commit"

# 2. 브랜치 이름을 main으로 설정합니다.
!git branch -M main

# 3. 원격으로 푸시를 시도합니다.
!git push -u origin main

[main (root-commit) 8e5c522] first commit
 1 file changed, 3 insertions(+)
 create mode 100644 README.md
fatal: could not read Username for 'https://github.com': No such device or address


In [79]:
!git remote remove origin

In [80]:
!git remote add origin git@github.com:dohb128/MachineLearning.git

In [85]:
!git branch -M main
!git branch

* main


In [88]:
!git add Toeic_Sample.ipynb
!git commit -m "first commit"
!git push -u origin main

[main 47d044b] first commit
 1 file changed, 1 insertion(+)
 create mode 100644 Toeic_Sample.ipynb
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [25]:
# 환경 설정 및 라이브러리 설치
!pip install pdfminer.six pandas
!pip install PyMuPDF pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 89.9 MB/s eta 0:00:00


In [53]:
# 정답 PDF 원시 텍스트 출력
ANSWER_PDF = 'toeic_part5_sample_answer.pdf' # 정답 파일 경로 추가

raw_answer_content = extract_text_with_pymupdf(ANSWER_PDF)
print("--- 정답 PDF 원시 텍스트 시작 ---")
print(raw_answer_content)

--- 정답 PDF 원시 텍스트 시작 ---
       TOEIC PART 5

      2196 Problems

       Answer Keys



           Compiled by krystal





Copyright (C) 2014 hackers all rights reserved.
[     1]                                                                                                                                                .

[       ]                                                                      [       ]

                                                                        ,               union          remark            ,   management


                              .
                                                                              [       ]
[       ]
                                                        (A)
to
                                       [     3]                                     to

                                                                              [       ]
                                                                           

# 문제 데이터 전처리

In [50]:
import re
import pandas as pd
from io import StringIO
import re
# (read_txt_file 함수와 raw_text 변수는 이미 정의되었다고 가정합니다.)

def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def parse_txt_by_block_final(raw_text):
    """
    대괄호 태그를 문제 ID로 사용하고, 옵션은 다음 보기 괄호 직전까지의 텍스트를 포착합니다.
    """

    # 1. 텍스트 클리닝: ID 태그 통일 및 공백 정규화
    cleaned_text = re.sub(r'\n', ' ', raw_text)
    cleaned_text = re.sub(r'\[\s*(\d+)\s*\]', r'[ID\1]', cleaned_text) # [ 1] -> [ID1]로 변환
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text).strip()

    data = []

    # 2. 정규표현식 패턴 (옵션 경계를 다음 괄호로 설정)
    pattern = re.compile(
        # 1. 문제 ID (그룹 1: ID)
        r'\[ID(\d+)\]\s*'
        # 2. Question Text (그룹 2): (A) 직전까지 포착
        # Question Text 내부에 섞여 들어간 불필요한 'X._______' 패턴을
        # Question Text를 포착하기 전에 제거하는 것이 더 안전합니다.
        r'(.*?)\s*'

        # 3. Option A (그룹 3): (B) 직전까지 포착
        # (A)와 (B) 사이의 모든 공백과 텍스트를 포착하며, (B)는 포착 그룹에 포함하지 않습니다.
        r'\(A\)\s*(.*?)\s*\(B\)'

        # 4. Option B (그룹 4): (C) 직전까지 포착
        r'\s*(.*?)\s*\(C\)'

        # 5. Option C (그룹 5): (D) 직전까지 포착
        r'\s*(.*?)\s*\(D\)'

        # 6. Option D (그룹 6): 다음 ID 태그 또는 파일 끝 직전까지 포착
        r'\s*(.*?)\s*'

        # 다음 ID 태그 또는 파일 끝을 최종 경계로 사용
        r'(?=\s*\[ID\d+\]|\Z)',
        re.DOTALL
    )

    matches = pattern.finditer(cleaned_text)

    for match in matches:
        q_text_raw = match.group(2)

        # Question Text 내부에 섞여 들어간 불필요한 '1. To...' 같은 문제 번호를 제거합니다.
        # (이전 문제에서 다음 문제 문장만 추출하기 위함)
        q_text_cleaned = re.sub(r'^\s*\d+\.\s*', '', q_text_raw).strip()

        data.append({
            'Question_ID': int(match.group(1).strip()),
            'Question_Text': q_text_cleaned,
            'Option_A': match.group(3).strip(),
            'Option_B': match.group(4).strip(),
            'Option_C': match.group(5).strip(),
            'Option_D': match.group(6).strip()
        })

    return pd.DataFrame(data)

# --- 실행 블록 ---

raw_text = read_txt_file('/content/toeic5_sample.txt')
problems_df = parse_txt_by_block_final(raw_text)

print(f"✅ 추출 성공: 총 {len(problems_df)}개의 문제 추출 완료.")
print("\n--- 문제 데이터프레임 최종 미리보기 (단어 옵션) ---")
print(problems_df[['Question_ID', 'Question_Text', 'Option_A', 'Option_B', 'Option_C', 'Option_D']].head())
problems_df.to_csv('toeic_sample_dataset.csv', index=False, encoding='utf-8-sig')


✅ 추출 성공: 총 53개의 문제 추출 완료.

--- 문제 데이터프레임 최종 미리보기 (단어 옵션) ---
   Question_ID                                      Question_Text  Option_A  \
0            1  To _______ the repair job on time, the enginee...  finished   
1            2  The union leader plans to _______ his remarks ...   address   
2            3  _______ to secure your export products, we rec...  In order   
3            4  The consultant was impressed that the draftsme...  standard   
4            5  Jerry Cunningham will ------- the names of res...  announce   

     Option_B     Option_C  Option_D  
0   finishing     finishes    finish  
1       exalt    broadcast      send  
2     So that       Though   Because  
3  correction  supervision  accuracy  
4     diffuse        trade   command  


# 정답 데이터 전처리

In [63]:
import re
import pandas as pd
from io import StringIO
import os # 파일 경로 확인용

# --- 정답 키 파싱 함수 ---

def read_txt_file(file_path):
    """안전한 인코딩으로 TXT 파일을 읽습니다."""
    encodings = ['utf-8', 'euc-kr', 'latin-1']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                return f.read()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"오류: {file_path}를 모든 인코딩으로 디코딩할 수 없습니다.")

def parse_clean_answer_key(raw_answer_text):
    """
    [ ID ]\n(Label) 패턴에서 ID와 Label을 추출하여 DataFrame을 만듭니다.
    """
    # 텍스트 클리닝: 모든 줄바꿈과 공백을 하나의 공백으로 치환합니다.
    cleaned_text = re.sub(r'\s+', ' ', raw_answer_text).strip()

    # 정규표현식 패턴: [ 1 ] (D) 패턴을 포착
    # 그룹 1: ID, 그룹 2: Label
    pattern = re.compile(r'\[\s*(\d+)\s*\]\s*\(([A-D])\)')

    # 모든 정답 쌍을 찾습니다.
    answer_matches = pattern.findall(cleaned_text)

    # DataFrame으로 변환합니다.
    return pd.DataFrame({
        'Question_ID': [int(qid) for qid, label in answer_matches],
        'Answer_Label': [label for qid, label in answer_matches]
    })

# --- 실행 블록 ---

# 1. 정답 데이터 읽기 및 파싱
answer_file_path = '/content/toeic5_sample_answer.txt'

try:
    # 🚨 파일 존재 여부 확인 (Colab 환경에서 필요)
    if not os.path.exists(answer_file_path):
         print(f"🚨 오류: 정답 파일 '{answer_file_path}'이 현재 경로에 없습니다. 파일을 업로드해주세요.")
    else:
         raw_text = read_txt_file(answer_file_path)


    answers_df = parse_clean_answer_key(raw_text)

    # 2. 딥러닝 학습용 정수 레이블 추가
    label_to_int = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    answers_df['Target_Class'] = answers_df['Answer_Label'].map(label_to_int)

    print(f"✅ 정답 데이터 추출 성공: 총 {len(answers_df)}개의 정답 레이블 확보.")

    print("\n--- 최종 정답 데이터프레임 (`answers_df`) ---")
    print(answers_df.head())
    print("\n--- 마지막 정답 확인 ---")
    print(answers_df.tail())

    # 3. CSV로 저장 (선택 사항)
    answers_df.to_csv('clean_answer_keys.csv', index=False, encoding='utf-8-sig')
    print("\n✅ 'clean_answer_keys.csv' 파일 저장 완료.")

except Exception as e:
    print(f"🚨 추출 중 치명적인 오류 발생: {e}")

✅ 정답 데이터 추출 성공: 총 53개의 정답 레이블 확보.

--- 최종 정답 데이터프레임 (`answers_df`) ---
   Question_ID Answer_Label  Target_Class
0            1            D             3
1            2            A             0
2            3            A             0
3            4            C             2
4            5            A             0

--- 마지막 정답 확인 ---
    Question_ID Answer_Label  Target_Class
48           49            B             1
49           50            A             0
50           51            D             3
51           52            A             0
52           53            C             2

✅ 'clean_answer_keys.csv' 파일 저장 완료.


# 문제-정답 매칭

In [64]:
import pandas as pd

# --- 문제 데이터프레임 (problems_df) 로드 ---
try:
    problems_df = pd.read_csv('/content/toeic_sample_dataset.csv')
except FileNotFoundError:
    # 파일이 저장되지 않은 경우, 사용자 제공 데이터를 기반으로 임시 DF 생성
    print("경고: 문제 CSV 파일이 없어 임시 데이터를 생성합니다. 실제 데이터와 다를 수 있습니다.")
    problems_data = {
        'Question_ID': [1, 2, 3, 4, 5],
        'Question_Text': ['To _______ the repair job on time, the enginee...', 'The union leader plans to _______ his remarks ...', '_______ to secure your export products, we rec...', 'The consultant was impressed that the draftsme...', 'Jerry Cunningham will ------- the names of res...'],
        'Option_A': ['finished', 'address', 'In order', 'standard', 'announce'],
        'Option_B': ['finishing', 'exalt', 'So that', 'correction', 'diffuse'],
        'Option_C': ['finishes', 'broadcast', 'Though', 'supervision', 'trade'],
        'Option_D': ['finish', 'send', 'Because', 'accuracy', 'command']
    }
    problems_df = pd.DataFrame(problems_data)


# --- 정답 데이터프레임 (answers_df) 로드 ---
try:
    answers_df = pd.read_csv('/content/toeic_sample_dataset_answer.csv')
except FileNotFoundError:
    # 파일이 저장되지 않은 경우, 사용자 제공 데이터를 기반으로 임시 DF 생성
    print("경고: 정답 CSV 파일이 없어 임시 데이터를 생성합니다. 실제 데이터와 다를 수 있습니다.")
    answers_data = {
        'Question_ID': list(range(1, 54)),
        'Answer_Label': ['D', 'A', 'A', 'C', 'A'] * 10 + ['B', 'A', 'C'], # 53개 맞추기 위해 임의로 채움
        'Target_Class': [3, 0, 0, 2, 0] * 10 + [1, 0, 2]
    }
    answers_df = pd.DataFrame(answers_data)
    answers_df = answers_df.iloc[:53]


# --- 최종 병합 실행 ---

# Question_ID를 기준으로 두 DataFrame을 내부 병합합니다. (모든 문제에 정답이 있다고 가정)
final_dataset_df = problems_df.merge(answers_df, on='Question_ID', how='inner')

# 최종 확인 및 출력
print(f"✅ 최종 데이터셋 생성 완료: 총 {len(final_dataset_df)}개의 문제-정답 쌍 확보.")

print("\n--- 최종 학습 데이터셋 미리보기 ---")
print(final_dataset_df[['Question_ID', 'Question_Text', 'Option_A', 'Answer_Label', 'Target_Class']].head())

# 최종 CSV 파일 저장
final_dataset_df.to_csv('toeic_final_dataset_merged.csv', index=False, encoding='utf-8-sig')
print("\n✅ 최종 CSV 파일 'toeic_final_dataset_merged.csv' 저장 완료. 이제 PyTorch 학습을 시작할 수 있습니다. 🚀")

✅ 최종 데이터셋 생성 완료: 총 53개의 문제-정답 쌍 확보.

--- 최종 학습 데이터셋 미리보기 ---
   Question_ID                                      Question_Text  Option_A  \
0            1  To _______ the repair job on time, the enginee...  finished   
1            2  The union leader plans to _______ his remarks ...   address   
2            3  _______ to secure your export products, we rec...  In order   
3            4  The consultant was impressed that the draftsme...  standard   
4            5  Jerry Cunningham will ------- the names of res...  announce   

  Answer_Label  Target_Class  
0            D             3  
1            A             0  
2            A             0  
3            C             2  
4            A             0  

✅ 최종 CSV 파일 'toeic_final_dataset_merged.csv' 저장 완료. 이제 PyTorch 학습을 시작할 수 있습니다. 🚀
